In [17]:
import torch

class Node:
    def __init__(self, node_id: str, node_type: str, cores: int = 1, cuda_streams: int = 1):
        """
        Initialize a computing node for executing tasks.

        Args:
            node_id (str): Unique identifier for the node.
            node_type (str): Type of node - either "CPU" or "GPU".
            cores (int): Number of CPU cores (for CPU nodes).
            cuda_streams (int): Number of CUDA streams (for GPU nodes only).
        """
        self.node_id = node_id
        self.node_type = node_type
        self.cores = cores
        self.cuda_streams = cuda_streams if node_type == "GPU" else None
        self.assigned_stages = []  # Stages assigned to this node

    def add_stage(self, stage):
        """
        Add a stage to this node.

        Args:
            stage (Stage): Stage to be added to the node.
        """
        self.assigned_stages.append(stage)

    def execute_stage(self, stage):
        """
        Execute a stage on this node.

        Args:
            stage (Stage): Stage to be executed.

        Returns:
            float: Execution time of the stage.
        """
        print(f"Executing stage {stage.stage_id} on node {self.node_id} (type: {self.node_type})")

        # Execute stage on CPU or GPU
        if self.node_type == "CPU":
            # Simulate CPU execution using PyTorch CPU tensor operations
            # tensor = torch.ones((100, 100), device="cpu")
            # priorty setting of CPU cores and processes or threads on each cpu cores

            # This is stage excuetion
            for _ in range(stage.layers):  # Each layer represents a unique operation
                # excuete layers
                pass

        elif self.node_type == "GPU":

            # Simulate GPU execution using PyTorch GPU tensor operations
            stream = torch.cuda.Stream(device="cuda:0", priority=0)  # Low priority stream for BE
            with torch.cuda.stream(stream):
                # stage function
                pass
            torch.cuda.synchronize()  # Ensure all GPU operations are complete

        execution_time = stage.compute_execution_time(self)  # Calculate execution time for profiling
        return execution_time

    # representation when the object is printed.
    def __repr__(self):
        return f"Node(id={self.node_id}, type={self.node_type}, cores={self.cores}, cuda_streams={self.cuda_streams})"


In [18]:
class Stage:
    def __init__(self, stage_id: str, layers: int, node: Node, task_type: str = "BE", priority: int = 0):
        """
        Initialize a stage of a task that will be executed on a node.

        Args:
            stage_id (str): Unique identifier for the stage.
            layers (int): Number of layers in the stage.
            node (Node): Node on which this stage will be executed.
            task_type (str): Type of task - either "RT" (Real-Time) or "BE" (Best-Effort).
            priority (int): Priority of the stage (higher value means higher priority).
        """
        self.stage_id = stage_id
        self.layers = layers
        self.node = node
        self.task_type = task_type
        self.priority = priority

    def compute_execution_time(self, node: Node):
        """
        Compute the execution time for the stage on the given node.

        Args:
            node (Node): Node on which the stage is being executed.

        Returns:
            float: Simulated execution time.
        """
        base_time = 1.0  # Base execution time per layer in seconds (this is a placeholder)
        execution_time = base_time * self.layers / node.cores
        return execution_time

    def __repr__(self):
        return f"Stage(id={self.stage_id}, layers={self.layers}, node={self.node.node_id}, type={self.task_type}, priority={self.priority})"


In [19]:
from typing import List
class Task:
    def __init__(self, task_id: str, stages: List[Stage], task_type: str = "BE", period: float = 1.0, deadline: float = 1.0):
        """
        Initialize a DNN inference task composed of multiple stages.

        Args:
            task_id (str): Unique identifier for the task.
            stages (List[Stage]): List of stages that make up this task.
            task_type (str): Type of task - either "RT" (Real-Time) or "BE" (Best-Effort).
            period (float): Minimum inter-arrival time between instances of the task.
            deadline (float): Deadline for the task to complete.
        """
        self.task_id = task_id
        self.stages = stages
        self.task_type = task_type
        self.period = period
        self.deadline = deadline

    def assign_node_to_stages(self):
        """
        Assign the corresponding node to each stage in the task.
        """
        for stage in self.stages:
            stage.node.add_stage(stage)

    def __repr__(self):
        return f"Task(id={self.task_id}, type={self.task_type}, stages={self.stages}, deadline={self.deadline})"


In [20]:
class TaskSet:
    def __init__(self, task_set_id: str, tasks: List[Task], nodes: List[Node]):
        """
        Initialize a set of tasks to be executed on available nodes.

        Args:
            task_set_id (str): Unique identifier for the task set.
            tasks (List[Task]): List of tasks in the task set.
            nodes (List[Node]): List of nodes available for task execution.
        """
        self.task_set_id = task_set_id
        self.tasks = tasks
        self.nodes = nodes

    def allocate_tasks(self):
        """
        Allocate tasks to nodes based on task stages.
        """
        for task in self.tasks:
            task.assign_node_to_stages()

    def execute_task_set(self):
        """
        Execute all tasks in the task set on their assigned nodes.
        """
        print(f"Executing TaskSet {self.task_set_id}")
        for node in self.nodes:
            for stage in node.assigned_stages:
                execution_time = node.execute_stage(stage)
                print(f"  Stage {stage.stage_id} on Node {node.node_id} executed in {execution_time:.2f} seconds")

    def __repr__(self):
        return f"TaskSet(id={self.task_set_id}, tasks={self.tasks}, nodes={self.nodes})"


In [21]:
'''
Sample Script for TaskSet Execution

User would be creating something similar to this except for the stage object creation which
would be handled by the Scheduler
'''



# Step 1: Define nodes
cpu_node_1 = Node(node_id="CPU_1", node_type="CPU", cores=4)
cpu_node_2 = Node(node_id="CPU_2", node_type="CPU", cores=4)

# This would be for CUDA
gpu_node = Node(node_id="GPU_1", node_type="GPU", cores=1, cuda_streams=2)

# Step 2: Define stages with associated nodes
stage1 = Stage(stage_id="S1", layers=3, node=cpu_node_1, task_type="RT", priority=10)
stage2 = Stage(stage_id="S2", layers=4, node=cpu_node_2, task_type="BE", priority=5)
stage3 = Stage(stage_id="S3", layers=2, node=gpu_node, task_type="RT", priority=5)

# Step 3: Create tasks with the defined stages
task1 = Task(task_id="RT_Task", stages=[stage1], task_type="RT", period=20.0, deadline=10.0)
task2 = Task(task_id="BE_Task", stages=[stage2], task_type="BE", period=30.0, deadline=15.0)
task3 = Task(task_id="RT_Task_2", stages=[stage3], task_type="RT", period=20.0, deadline=10.0)

# Step 4: Initialize TaskSet with tasks and nodes
task_set = TaskSet(task_set_id="TS1", tasks=[task1, task2,task3], nodes=[cpu_node_1, cpu_node_2,gpu_node])

# Step 5: Allocate tasks to their respective nodes
task_set.allocate_tasks()

# Step 6: Execute the task set, running each stage on its assigned node
task_set.execute_task_set()


Executing TaskSet TS1
Executing stage S1 on node CPU_1 (type: CPU)
  Stage S1 on Node CPU_1 executed in 0.75 seconds
Executing stage S2 on node CPU_2 (type: CPU)
  Stage S2 on Node CPU_2 executed in 1.00 seconds
Executing stage S3 on node GPU_1 (type: GPU)
  Stage S3 on Node GPU_1 executed in 2.00 seconds
